In [6]:
import numpy as np
import pydub
import requests
import os
import json

### mp3 to nparray
You need ffmpeg installed and added to path to use pydub

In [2]:
def mp3_to_np(mp3_file, normalized=False):
    """MP3 to numpy array"""
    a = pydub.AudioSegment.from_mp3(mp3_file)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

def np_to_mp3(dest_file, sr, x, normalized=False):
    """numpy array to MP3"""
    channels = 2 if (x.ndim == 2 and x.shape[1] == 2) else 1
    if normalized:  # normalized array - each item should be a float in [-1, 1)
        y = np.int16(x * 2 ** 15)
    else:
        y = np.int16(x)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=sr, sample_width=2, channels=channels)
    song.export(dest_file, format="mp3", bitrate="320k")

#### Download select mp3 files for local machine testing

In [17]:
SELECT_SPECIES = ["Pale-legged_Leaf_Warbler", "Black-crested_Bulbul", "Blue_Whistling_Thrush", 
                  "Orange-bellied_Flowerpecker", "Great_Slaty_Woodpecker"]
PATH = "./data/mp3/"

for filename in os.listdir("./data"):
    if filename[:-5] in SELECT_SPECIES:
        with open("./data/" + filename, 'r') as speciesFile:
            species_data = json.load(speciesFile)
            #for i in range(0, len(species_data)):
            for i in range(0, 3):
                d = species_data[i]
                if d["file-name"].endswith(".mp3"):
                    mp3_url = d["file"]
                    r = requests.get(mp3_url, allow_redirects=True)
                    open(PATH + filename[:-5] + f"_{i}.mp3", 'wb').write(r.content)

In [19]:
numpy_audio_sample = mp3_to_np("data/mp3/Blue_Whistling_Thrush_0.mp3")